In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import nrrd
print("------------------------------------------------------------------------------------------------")
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print("------------------------------------------------------------------------------------------------")

2024-07-02 09:59:57.143085: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 09:59:57.953945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


------------------------------------------------------------------------------------------------
2.16.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
------------------------------------------------------------------------------------------------


In [2]:

def encoder_block(inputs, output_channels, lastlayer=False):
    """
    Two 3x3x3 convolutions with batch normalization and ReLU activation
    2x2x2 max pool
    """

    # 3x3x3 convolutions with ReLU activation
    x = tf.keras.layers.Conv3D(int(output_channels/2), kernel_size=3, strides=1, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv3D(output_channels, kernel_size=3, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # 2x2x2 max pool

    if not lastlayer:
        x_maxPool = tf.keras.layers.MaxPool3D(pool_size=2, strides=2, padding = 'same')(x)
    else:
        x_maxPool = x

    return x, x_maxPool

def decoder_block(inputs, skip_features, output_channels):

    # Upsampling with 2x2x2 filter
    x = tf.keras.layers.Conv3DTranspose(output_channels*2, kernel_size=2, strides=2, padding = 'same')(inputs)

# Concatenate the skip features
    x = tf.keras.layers.Concatenate()([x, skip_features])

    # 2 convolutions with 3x3 filter, batch normalization, ReLU activation
    x = tf.keras.layers.Conv3D(output_channels, kernel_size=3, strides=1, padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv3D(output_channels, kernel_size=3, strides=1, padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    return x

def unet_3D():
    inputs = tf.keras.Input(shape=(64, 64, 64, 1,))

    e1_skip, e1_maxpool = encoder_block(inputs, 64)
    e2_skip, e2_maxpool = encoder_block(e1_maxpool, 128)
    e3_skip, e3_maxpool = encoder_block(e2_maxpool, 256)
    _, e4 = encoder_block(e3_maxpool, 512, True)

    decoder1 = decoder_block(e4, e3_skip, 256)
    decoder2 = decoder_block(decoder1, e2_skip, 128)
    decoder3 = decoder_block(decoder2, e1_skip, 64)

    outputs = tf.keras.layers.Conv3D(1, 1, strides = 1)(decoder3)
    # outputs = tf.keras.layers.Conv3D(2, 1, strides = 1)(decoder3)
    # outputs = tf.keras.layers.Reshape((64*64*64, 2))(outputs)
    outputs = tf.keras.layers.Activation('sigmoid')(outputs)

    model = tf.keras.models.Model(inputs = inputs,  outputs = outputs,  name = 'Unet3D')

    return model
    

In [3]:
def iou(y_true, y_pred, smooth=0.000000001):
    # yt = K.argmax(y_true, axis=2)
    # yp = K.argmax(y_pred, axis=2)
    # print(y_pred)
    #yp = y_pred[0]
    # yp[yp>=0.5]=1
    # yp[yp<0.5]=0
    yp = y_pred
    yp = tf.where(yp >= 0.5, tf.ones_like(yp), yp)
    yp = tf.where(yp < 0.5, tf.zeros_like(yp), yp)
    yp = K.cast(yp, np.float32)

    yt = K.cast(y_true, np.float32)

    # print(yt.shape)
    # print(yp.shape)
    
    intersection = K.sum(yt * yp)
    union = K.sum(yt) + K.sum(yp)
    # intersection = K.sum(yt * yp, axis=1)
    # union = K.sum(yt, axis=1) + K.sum(yp, axis=1)
    return (intersection + smooth) / (union-intersection+smooth)

In [4]:
model = unet_3D()
# model.summary()

print("compiling model")
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='dice')#, metrics=[iou])

2024-07-02 10:00:01.304173: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10089 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:65:00.0, compute capability: 8.9


compiling model


In [5]:
import os
import matplotlib.pyplot as plt

In [6]:
import json
# Get Patch Coordinates

with open("patch_coords.json", 'r') as f:
    p_coords = json.load(f)

print(p_coords)

[[512, 512, 211], [1, 311, 375, 179, 243, 27, 91], [2, 243, 307, 139, 203, 33, 97], [3, 209, 273, 143, 207, 40, 104], [4, 188, 252, 175, 239, 54, 118], [5, 220, 284, 175, 239, 66, 130], [6, 168, 232, 137, 201, 88, 152], [7, 311, 375, 145, 209, 90, 154], [8, 152, 216, 172, 236, 103, 167], [9, 186, 250, 175, 239, 118, 182], [10, 124, 188, 204, 268, 121, 185], [11, 328, 392, 177, 241, 122, 186], [12, 276, 340, 162, 226, 122, 186], [13, 173, 237, 207, 271, 129, 193]]


In [7]:
cp_number = "{:04d}".format(140)
print(cp_number)
model.load_weights("./checkpoints/cp-0140.weights.h5")

number = 18
reformation = np.zeros(shape=(p_coords[0]))

for patch in range(1, len(p_coords)):
    print(p_coords[patch])    
    try:
        X, _ = nrrd.read("./test_data/inputs/18_volume_"+str(p_coords[patch][0])+".nrrd")
        X = np.array([X]).astype(np.float32)
        X = np.expand_dims(X, -1)
        
        y = model.predict(X, verbose=0)
        y = y[0]
        y[y>=0.5]=1
        y[y<0.5]=0
        y = y[...,0]
        print(y.shape)
        reformation[p_coords[patch][1]:p_coords[patch][2], p_coords[patch][3]:p_coords[patch][4], p_coords[patch][5]:p_coords[patch][6]] += y
    except:
        pass
        
reformation[reformation > 1] = 1

nrrd.write("./test_data/reform/predictions/"+str(number)+"_checkpoint_"+str(cp_number)+".nrrd", reformation)



0140
[1, 311, 375, 179, 243, 27, 91]


/home/kevin.wang/miniconda3/envs/cactas/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 130 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1719928809.067070  141793 service.cc:145] XLA service 0x72c0b0016480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719928809.067125  141793 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-07-02 10:00:09.099001: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-02 10:00:09.207408: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
I0000 00:00:1719928811.647600  141793 device_compiler.h:188] Compiled cluster using XLA!  Thi

(64, 64, 64)
[2, 243, 307, 139, 203, 33, 97]
(64, 64, 64)
[3, 209, 273, 143, 207, 40, 104]
(64, 64, 64)
[4, 188, 252, 175, 239, 54, 118]
(64, 64, 64)
[5, 220, 284, 175, 239, 66, 130]
(64, 64, 64)
[6, 168, 232, 137, 201, 88, 152]
(64, 64, 64)
[7, 311, 375, 145, 209, 90, 154]
(64, 64, 64)
[8, 152, 216, 172, 236, 103, 167]
(64, 64, 64)
[9, 186, 250, 175, 239, 118, 182]
(64, 64, 64)
[10, 124, 188, 204, 268, 121, 185]
(64, 64, 64)
[11, 328, 392, 177, 241, 122, 186]
(64, 64, 64)
[12, 276, 340, 162, 226, 122, 186]
(64, 64, 64)
[13, 173, 237, 207, 271, 129, 193]
(64, 64, 64)


In [8]:
np.unique(reformation)

array([0., 1.])

In [9]:
def compute_iou(y_true, y_pred, smooth=0.000000001):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred)
    return (intersection + smooth) / (union-intersection+smooth)

In [10]:
gt, _ = nrrd.read("./test_data/reform/predictions/"+str(number)+"_gt.nrrd")

In [11]:
compute_iou(gt, reformation)

0.5149051490519287

In [12]:
number = 18
gt, _ = nrrd.read("./test_data/reform/predictions/"+str(number)+"_gt.nrrd")

best_cp = 0
best_iou = 0

for cp in range(1, 200):
    cp_number = "{:04d}".format(cp)
    print(cp_number)
    model.load_weights("./checkpoints/cp-"+str(cp_number)+".weights.h5")
    
    reformation = np.zeros(shape=(p_coords[0]))
    
    for patch in range(1, len(p_coords)): 
        try:
            X, _ = nrrd.read("./test_data/inputs/18_volume_"+str(p_coords[patch][0])+".nrrd")
            X = np.array([X]).astype(np.float32)
            X = np.expand_dims(X, -1)
            
            y = model.predict(X, verbose=0)
            y = y[0]
            y[y>=0.5]=1
            y[y<0.5]=0
            y = y[...,0]
            reformation[p_coords[patch][1]:p_coords[patch][2], p_coords[patch][3]:p_coords[patch][4], p_coords[patch][5]:p_coords[patch][6]] += y
        except:
            pass
            
    reformation[reformation > 1] = 1
    
    nrrd.write("./test_data/reform/predictions/"+str(number)+"_checkpoint_"+str(cp_number)+".nrrd", reformation)
    iou = compute_iou(gt, reformation)
    if iou > best_iou:
        best_iou = iou
        best_cp = cp
    print("IOU: " + str(iou))


0001
IOU: 0.13712910986436477
0002
IOU: 0.24081313526251694
0003
IOU: 0.30680948737619984
0004
IOU: 0.18682235196064484
0005
IOU: 0.17998244073820896
0006
IOU: 0.2188524590170337
0007
IOU: 0.29619260918292484
0008
IOU: 0.052478134111707986
0009
IOU: 0.3401988636368322
0010
IOU: 0.305005820722174
0011
IOU: 0.1279317697237442
0012
IOU: 0.21160042964638925
0013
IOU: 0.3059033989272756
0014
IOU: 0.35094962840681176
0015
IOU: 0.363414634146859
0016
IOU: 0.23772102161175077
0017
IOU: 0.323140495868328
0018
IOU: 0.29395085066229304
0019
IOU: 0.30963302752346356
0020
IOU: 0.32136602451898305
0021
IOU: 0.4092219020178586
0022
IOU: 0.14898177920777173
0023
IOU: 0.2657777777784304
0024
IOU: 0.23781009409817125
0025
IOU: 0.2972195589651992
0026
IOU: 0.19201725997929664
0027
IOU: 0.2726358148900678
0028
IOU: 0.12651265126608746
0029
IOU: 0.3391472868223458
0030
IOU: 0.4596375617796873
0031
IOU: 0.25643564356509263
0032
IOU: 0.3414634146347002
0033
IOU: 0.2585034013612648
0034
IOU: 0.282907662083219

In [14]:
print(best_iou, best_cp)

0.5973568281941873 101
